In [1]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
import xgboost as xgb
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold,train_test_split
import csv
import pandas as pd
import pickle
import numpy as np
n_classes=27
categories_dict = {'Accessories': 0, 'Apparel Set': 1, 'Bags': 2, 'Belts': 3, 'Bottomwear': 4, 'Cufflinks': 5, 'Dress': 6, 'Eyewear': 7, 'Flip Flops': 8, 'Fragrance': 9, 'Free Gifts': 10, 'Headwear': 11, 'Innerwear': 12, 'Jewellery': 13, 'Lips': 14, 'Loungewear and Nightwear': 15, 'Makeup': 16, 'Nails': 17, 'Sandal': 18, 'Saree': 19, 'Scarves': 20, 'Shoes': 21, 'Socks': 22, 'Ties': 23, 'Topwear': 24, 'Wallets': 25, 'Watches': 26}
X = np.load('/content/drive/MyDrive/CS480/Final/TRAINX.npy')
y = np.load('/content/drive/MyDrive/CS480/Final/TRAINY.npy')

In [6]:
x_train,y_train,x_test,y_test = train_test_split(X,y,test_size=0.20)
kfolds = 3
ntrain = x_train.shape[0]
ntest = x_test.shape[0]

class MLP(object):
    def __init__(self):
        self.f = MLPClassifier(hidden_layer_sizes=(512,512),early_stopping=True,verbose=False,max_iter=1)
    def train(self, x_train, y_train):
        self.f.fit(x_train,y_train)
    def predict(self, x):
        return self.f.predict_proba(x)

def trainclf(clf):
    oof_train = np.zeros((len(x_train),n_classes))
    oof_test = np.zeros((kfolds, len(x_test),n_classes)) # test pred per fold
    kf = KFold(n_splits = kfolds, shuffle=False)
    for i, (train_index, test_index) in enumerate(kf.split(X)):
        x_tr = X[train_index]
        y_tr = y[train_index]
        x_te = X[test_index]
        y_te = y[test_index]
        
        clf.train(x_tr, y_tr)
        oof_train[test_index] = clf.predict(x_te)
        oof_test[i, :] = clf.predict(x_test)
    oof_test = oof_test.mean(axis=0)
    return oof_train, oof_test

In [ ]:
m1 = MLP()
m2 = MLP()
m3 = MLP()
tr1, te1 = trainclf(m1)
tr2, te2 = trainclf(m2)
tr3, te3 = trainclf(m3)

In [ ]:
x_tr_lg = np.concatenate((tr1, tr2, tr3), axis=1)
x_te_lg = np.concatenate((te1, te2, te3), axis=1)

In [8]:
lg = LogisticRegression()
lg.fit(x_tr_lg,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [9]:
ypred = lg.predict(x_te_lg)
p = [round(v) for v in ypred]
accuracy = accuracy_score(y_test, p)
accuracy

0.9292583687812096

In [9]:
%%capture
!pip3 install seaborn
#@title Load the Universal Sentence Encoder's TF Hub module
from absl import logging

import tensorflow as tf

import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns

module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" #@param ["https://tfhub.dev/google/universal-sentence-encoder/4", "https://tfhub.dev/google/universal-sentence-encoder-large/5"]
model = hub.load(module_url)
print ("module %s loaded" % module_url)
def embed(input):
  return model(input)

In [40]:
Categories = ['Accessories', 'Apparel Set', 'Bags', 'Belts', 'Bottomwear', 'Cufflinks', 'Dress', 'Eyewear', 'Flip Flops', 'Fragrance', 'Free Gifts', 'Headwear', 'Innerwear', 'Jewellery', 'Lips', 'Loungewear and Nightwear', 'Makeup', 'Nails', 'Sandal', 'Saree', 'Scarves', 'Shoes', 'Socks', 'Ties', 'Topwear', 'Wallets', 'Watches']
def get_preds(path,x_test):
    file = open(path,'r')
    df = pd.read_csv(file)
    preds = []
    for i, row in df.iterrows():
        id = row['id']
        enc = [x_test[i]]
        e1 = mlp1.predict(enc)
        e2 = mlp2.predict(enc)
        e3 = mlp3.predict(enc)
        enc = np.concatenate((e1,e2,e3),axis=1)
        pred = lg.predict(enc)[0]
        cat = Categories[pred]
        preds.append([id,cat])
        if i%1000==0: print(i)
    return preds

mlp4.csv is 4 folds of 4 mlp models 512 units hidden units 2 hidden layers
92.88884778990197

5 folds, 2 hidden layers of 512 units, with 2 models = 92.88422415387461



In [49]:
test_path = '/content/drive/MyDrive/CS480/Final/test.csv'
p = get_preds(test_path,XTEST)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000


In [16]:
with open("/content/drive/MyDrive/CS480/MLP3.csv","w") as csvFile:
    print("id,category",file=csvFile)
    for id,cat in p:
        print(str(id)+","+cat,file=csvFile)

In [52]:
lbls = [l[1] for l in p]
from collections import Counter
Counter(lbls)

Counter({'Accessories': 15,
         'Apparel Set': 5,
         'Bags': 1295,
         'Belts': 31,
         'Bottomwear': 4391,
         'Dress': 73,
         'Eyewear': 244,
         'Flip Flops': 293,
         'Fragrance': 171,
         'Free Gifts': 5,
         'Innerwear': 363,
         'Jewellery': 40,
         'Lips': 81,
         'Loungewear and Nightwear': 64,
         'Makeup': 54,
         'Nails': 81,
         'Sandal': 570,
         'Saree': 58,
         'Scarves': 4,
         'Shoes': 12124,
         'Socks': 204,
         'Ties': 1,
         'Topwear': 1070,
         'Wallets': 23,
         'Watches': 368})